# Exploratory Data Analysis (EDA)

This notebook contains the exploratory data analysis for the AIAP 21 Technical Assessment.

## Overview
This notebook will analyze the gas monitoring data to understand patterns, trends, and insights.

## Data Source
- Database: `data/gas_monitoring.db`
- Note: The database file should not be uploaded to the repository


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
from datetime import datetime

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")


In [ ]:
# Connect to the database
db_path = 'data/gas_monitoring.db'
conn = sqlite3.connect(db_path)

# Get table names
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Available tables:", [table[0] for table in tables])

# TODO: Load and explore the data based on the actual table structure


## Data Loading and Initial Exploration

This section will load the data from the database and perform initial exploration.


In [ ]:
# Load data from database
db_path = 'data/gas_monitoring.db'
conn = sqlite3.connect(db_path)

# Get table names
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Available tables:", [table[0] for table in tables])

# Load data from each table
dataframes = {}
for table in tables:
    table_name = table[0]
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql_query(query, conn)
    dataframes[table_name] = df
    print(f"\n{table_name} shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")

# Display first few rows of each table
for name, df in dataframes.items():
    print(f"\n=== {name.upper()} - First 5 rows ===")
    display(df.head())


In [ ]:
# Data quality assessment
print("🔍 DATA QUALITY ASSESSMENT")
print("=" * 50)

for name, df in dataframes.items():
    print(f"\n📊 {name.upper()}")
    print("-" * 30)
    print(f"Shape: {df.shape}")
    print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    print("\nData Types:")
    print(df.dtypes)
    
    print("\nMissing Values:")
    missing = df.isnull().sum()
    if missing.sum() > 0:
        print(missing[missing > 0])
    else:
        print("✅ No missing values")
    
    print("\nDuplicate Rows:")
    duplicates = df.duplicated().sum()
    print(f"Found {duplicates} duplicate rows")
    
    print("\nUnique Values per Column:")
    for col in df.columns:
        unique_count = df[col].nunique()
        print(f"  {col}: {unique_count} unique values")


In [ ]:
# Statistical analysis
print("📈 STATISTICAL ANALYSIS")
print("=" * 50)

for name, df in dataframes.items():
    print(f"\n📊 {name.upper()}")
    print("-" * 30)
    
    # Descriptive statistics for numerical columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print("Descriptive Statistics (Numerical Columns):")
        print(df[numeric_cols].describe())
    
    # Categorical analysis
    categorical_cols = df.select_dtypes(include=['object']).columns
    if len(categorical_cols) > 0:
        print("\nCategorical Columns Analysis:")
        for col in categorical_cols:
            print(f"\n{col}:")
            print(f"  Unique values: {df[col].nunique()}")
            print(f"  Most frequent: {df[col].value_counts().head(3).to_dict()}")
    
    print("\n" + "="*50)


In [ ]:
# Visualization
print("📊 CREATING VISUALIZATIONS")
print("=" * 50)

# Set up the plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Create visualizations for each table
for name, df in dataframes.items():
    print(f"\n📈 Creating visualizations for {name.upper()}")
    
    # Get numerical columns for plotting
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    
    if len(numeric_cols) > 0:
        # Create subplots for numerical columns
        n_cols = min(3, len(numeric_cols))
        n_rows = (len(numeric_cols) + n_cols - 1) // n_cols
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
        if n_rows == 1:
            axes = [axes] if n_cols == 1 else axes
        else:
            axes = axes.flatten()
        
        for i, col in enumerate(numeric_cols):
            if i < len(axes):
                # Histogram
                axes[i].hist(df[col].dropna(), bins=30, alpha=0.7, edgecolor='black')
                axes[i].set_title(f'{col} Distribution')
                axes[i].set_xlabel(col)
                axes[i].set_ylabel('Frequency')
        
        # Hide unused subplots
        for i in range(len(numeric_cols), len(axes)):
            axes[i].set_visible(False)
        
        plt.tight_layout()
        plt.show()
        
        # Correlation heatmap if multiple numerical columns
        if len(numeric_cols) > 1:
            plt.figure(figsize=(10, 8))
            correlation_matrix = df[numeric_cols].corr()
            sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
            plt.title(f'{name.upper()} - Correlation Matrix')
            plt.tight_layout()
            plt.show()
    
    # Categorical analysis
    categorical_cols = df.select_dtypes(include=['object']).columns
    if len(categorical_cols) > 0:
        for col in categorical_cols:
            if df[col].nunique() <= 20:  # Only plot if reasonable number of categories
                plt.figure(figsize=(12, 6))
                value_counts = df[col].value_counts()
                value_counts.plot(kind='bar')
                plt.title(f'{col} Value Counts')
                plt.xlabel(col)
                plt.ylabel('Count')
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.show()

print("\n✅ Visualizations completed!")


## Key Insights and Findings

This section will summarize the key insights from the EDA.


In [ ]:
# Key Insights and Findings
print("🔍 KEY INSIGHTS AND FINDINGS")
print("=" * 50)

for name, df in dataframes.items():
    print(f"\n📊 {name.upper()}")
    print("-" * 30)
    
    # Data overview
    print(f"Dataset contains {df.shape[0]:,} rows and {df.shape[1]} columns")
    
    # Missing data insights
    missing_data = df.isnull().sum()
    if missing_data.sum() > 0:
        print(f"⚠️  Missing data found in {missing_data[missing_data > 0].count()} columns")
        for col, missing_count in missing_data[missing_data > 0].items():
            percentage = (missing_count / len(df)) * 100
            print(f"   - {col}: {missing_count:,} missing ({percentage:.1f}%)")
    else:
        print("✅ No missing data")
    
    # Data quality insights
    duplicates = df.duplicated().sum()
    if duplicates > 0:
        print(f"⚠️  {duplicates:,} duplicate rows found")
    else:
        print("✅ No duplicate rows")
    
    # Numerical insights
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print(f"\n📈 Numerical Analysis ({len(numeric_cols)} columns):")
        for col in numeric_cols:
            col_data = df[col].dropna()
            if len(col_data) > 0:
                print(f"   - {col}:")
                print(f"     Range: {col_data.min():.2f} to {col_data.max():.2f}")
                print(f"     Mean: {col_data.mean():.2f}, Std: {col_data.std():.2f}")
                print(f"     Skewness: {col_data.skew():.2f}")
    
    # Categorical insights
    categorical_cols = df.select_dtypes(include=['object']).columns
    if len(categorical_cols) > 0:
        print(f"\n📋 Categorical Analysis ({len(categorical_cols)} columns):")
        for col in categorical_cols:
            unique_count = df[col].nunique()
            print(f"   - {col}: {unique_count} unique values")
            if unique_count <= 10:
                top_values = df[col].value_counts().head(3)
                print(f"     Top values: {top_values.to_dict()}")

print("\n🎯 RECOMMENDATIONS")
print("=" * 50)
print("1. Review missing data patterns and consider imputation strategies")
print("2. Investigate duplicate rows and determine if they should be removed")
print("3. Analyze numerical distributions for outliers and data quality issues")
print("4. Examine categorical variables for data consistency")
print("5. Consider feature engineering based on domain knowledge")
print("6. Plan further analysis based on business objectives")

# Close database connection
conn.close()
print("\n🔒 Database connection closed")
print("✅ EDA Analysis Complete!")
